### 📊 Group-wise Network Topological Analysis

This script performs topological analysis for each microbial co-abundance network, including:

- 🧩 Community detection (Louvain)
- 🧠 Node centrality metrics
- 🧱 Basic robustness indicators (e.g. global efficiency, average degree)

#### ✅ Outputs

For each group network (`*_edges.csv`), generates:


Containing the following columns:

| Column Name             | Description                                 |
|-------------------------|---------------------------------------------|
| Node                   | Taxon / genus name                          |
| Module                 | Louvain module assignment                   |
| Degree Centrality      | Influence based on direct connections       |
| Betweenness Centrality | Importance based on shortest paths          |
| Eigenvector Centrality | Influence weighted by influential neighbors |

#### 📈 Use Case

Use this output to:

- Identify network hubs or bridge taxa
- Study module-specific patterns
- Correlate network features with phenotype or environment

> Optional extensions:
> - Add network-level stats per group
> - Rank top `n` important taxa per metric


In [ ]:
import os
import networkx as nx
import community.community_louvain as community_louvain
import pandas as pd

# Input and output directory paths
input_dir = r'..group results'  # Replace with your edge list folder
output_dir = r'..genera results'  # Replace with your output path
os.makedirs(output_dir, exist_ok=True)

# Locate all group-specific edge files
network_files = [f for f in os.listdir(input_dir) if f.endswith('_edges.csv')]

# Loop over each group
for edges_file in network_files:
    group_name = edges_file.split('_edges.csv')[0]
    edges_file_path = os.path.join(input_dir, edges_file)
    output_file_path = os.path.join(output_dir, f'{group_name}_network_analysis.csv')

    # 1. Load edge list and build undirected graph
    edges_df = pd.read_csv(edges_file_path)
    G = nx.Graph()

    for _, row in edges_df.iterrows():
        G.add_edge(row['Node1'], row['Node2'], weight=row['Weight'])

    # 2. Ensure all weights are positive
    for u, v, data in G.edges(data=True):
        if data['weight'] < 0:
            data['weight'] = abs(data['weight'])

    # 3. Community detection using Louvain method
    partition = community_louvain.best_partition(G, weight='weight')
    nx.set_node_attributes(G, partition, 'Module')

    # 4. Node centrality metrics
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)

    # 5. Network robustness metrics
    efficiency = nx.global_efficiency(G)
    avg_degree = sum(dict(G.degree(weight='weight')).values()) / len(G.nodes())

    # 6. Build summary dataframe
    data = []
    for node in G.nodes():
        data.append({
            "Node": node,
            "Module": partition[node],
            "Degree Centrality": degree_centrality[node],
            "Betweenness Centrality": betweenness_centrality[node],
            "Eigenvector Centrality": eigenvector_centrality[node]
        })

    df = pd.DataFrame(data)
    df.to_csv(output_file_path, index=False)
    print(f"Saved network analysis for {group_name} to: {output_file_path}")

print("Network topology analysis completed for all groups.")
